![DLI Header](images/DLI_Header.png)

# Token Classification with Large Language Models #

## 01 - Named Entity Recognition with Pre-Trained Model ##

In this notebook, you will learn to use a pre-trained token classification model. Specifically, we will use a model for named entity recognition. NER, also referred to as entity chunking, identification or extraction, is the task of detecting and classifying key information (entities) in text. In other words, a NER model takes a piece of text as input and for each word in the text, the model identifies a category the word belongs to. For example, in a sentence: `Mary lives in Santa Clara and works at NVIDIA`, the model should detect that `Mary` is a person, `Santa Clara` is a location and `NVIDIA` is a company.

**Table of Contents**<br>
This notebook covers the below sections: 
* Project Overview
* Dataset
    * Download and Preprocess data
    * Labeling Data (OPTIONAL)
* Use Pre-Trained Model
    * Download Model
    * Make Predictions
    * Model Evaluation
* Fine-Tune a Pre-Trained Model

## Project Overview ##

<img src='images/workflow.png' width=1080>

## Dataset ##
For this notebook, we're going to use the [GMB (Groningen Meaning Bank)](http://www.let.rug.nl/bjerva/gmb/about.php) corpus for named entity recognition. GMB is a fairly large corpus with a lot of annotations. The data is labeled using the [IOB format](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)) (short for inside, outside, beginning), which means each annotation also needs a prefix of **I**, **O**, or **B**. 

The following classes appear in the dataset:
* **LOC** - Geographical Entity
* **ORG** - Organization
* **PER** - Person
* **GPE** - Geopolitical Entity
* **TIME** - Time indicator
* **ART** - Artifact
* **EVE** - Event
* **NAT** - Natural Phenomenon

_Note:_ GMB is not completely human annotated, and it’s not considered 100% correct. For this exercise, classes **ART**, **EVE**, and **NAT** were combined into a **MISC** class due to small number of examples for these classes.

For token classification tasks, NeMo requires the data to be in a specific format. Data needs to be split into  files: 
* `text.txt` and 
* `labels.txt`

Each line of the **text.txt** file contains text sequences, where words are separated with spaces, i.e.: `[WORD] [SPACE] [WORD] [SPACE] [WORD]`. The **labels.txt** file contains corresponding labels for each word in **text.txt**, the labels are separated with spaces, i.e.: `[LABEL] [SPACE] [LABEL] [SPACE] [LABEL]`.

For example: 
* **text.txt**
```
Jennifer is from New York City .
She likes ...
...
```
* **labels.txt**
```
B-PER O O B-LOC I-LOC I-LOC O
O O ...
...
```

### Download and Preprocess Data ###

In [5]:
# !pip install wget
import os
import wget

# set data path
DATA_DIR="data/GMB"

In [6]:
# check that data folder should contain 4 files
!ls -l $DATA_DIR

total 11140
-rw-r--r-- 1 kappa kappa      77 Aug 19 15:39 label_ids.csv
-rw-r--r-- 1 kappa kappa  407442 Aug 19 15:39 labels_dev.txt
-rw-r--r-- 1 kappa kappa 3169783 Aug 19 15:40 labels_train.txt
-rw-r--r-- 1 kappa kappa  891020 Aug 19 15:40 text_dev.txt
-rw-r--r-- 1 kappa kappa 6928251 Aug 19 15:40 text_train.txt


In [7]:
# preview data 
print('Text:')
!head -n 5 {DATA_DIR}/text_train.txt

print('Labels:')
!head -n 5 {DATA_DIR}/labels_train.txt

Text:
New Zealand 's cricket team has scored a morale-boosting win over Bangladesh in the first of three one-day internationals in New Zealand .
Despite Bangladesh 's highest total ever in a limited-overs match , the Kiwis were able to win the match by six wickets in Auckland .
Opening batsman Jamie How led all scorers with 88 runs as New Zealand reached 203-4 in 42.1 overs .
The score was in response to Bangladesh 's total of 201 all out in 46.3 overs .
Mohammad Ashraful led the visitors with 70 runs , including 10 fours and one six on the short boundaries of the Eden Park ground .
Labels:
B-LOC I-LOC O O O O O O O O O B-LOC O O B-TIME I-TIME I-TIME I-TIME O O B-LOC I-LOC O
O B-LOC O O O O O O O O O O B-GPE O O O O O O O O O O B-LOC O
O O B-PER I-PER O O O O O O O B-LOC I-LOC O O O O O O
O O O O O O B-LOC O O O O O O O O O O
B-PER I-PER O O O O O O O O O O O O O O O O O O O B-LOC I-LOC O O


### Labeling Data ###

If you have raw data, NeMo recommends using the [Datasaur](https://datasaur.ai/) labeling platform to apply labels to data. Datasaur was designed specifically for labeling text data and supports basic NLP labeling tasks such as Named Entity Recognition and text classification through advanced NLP tasks such as dependency parsing and coreference resolution. You can sign up for Datasaur for free at https://datasaur.ai/sign-up/. Once you upload a file, you can choose from multiple NLP project types and use the Datasaur interface to label the data. After labeling, you can export the labeled data using the conll_2003 format, which integrates directly with NeMo. A video walkthrough can be found [here](https://www.youtube.com/watch?v=I9WVmnnSciE).

## Use Pre-Trained Model ##
NeMo supports NER and other token-level classification tasks. These models typically comprise of a pre-trained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model followed by a token classification layer. We start by using a pre-trained model. The `TokenClassificationModel` inherits from `NLPModel` and has the below methods: 
* `TokenClassificationModel.add_predictions()`
* `TokenClassificationModel.evaluate_from_file()`

These are useful when making inference and evaluating model performance. Additional functionality of the `TokenClassificationModel` can be found in the [source code](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/nlp/models/token_classification/token_classification_model.py). 

### Download Pre-Trained Model ###

In [2]:
# !pip install git+https://github.com/NVIDIA/NeMo.git
# !pip install hydra-core
# !pip install pytorch_lightning
# !pip install einops
# !pip install transformers
# !pip install sentencepiece
# !pip install braceexpand
# !pip install webdataset
# !pip install h5py
# !pip install ijson
# !pip install pandas
# !pip install sacremoses
# !pip install matplotlib
# !pip install megatron
# !pip install megatron.core
# !pip install sacrebleu
# !pip install rouge_score
!pip install apex
from nemo.collections.nlp.models import TokenClassificationModel

# list available pre-trained models
for model in TokenClassificationModel.list_available_models():
    print(model)

ImportError: cannot import name 'UnencryptedCookieSessionFactoryConfig' from 'pyramid.session' (unknown location)

_Note:_ These are models trained for token classification. To get a list of all supported models, use `nemo.collections.nlp.modules.get_pretrained_lm_models_list(include_external=True)`. The list of pre-trained models is expected to change as they become available. 

In [ ]:
# download and load the pre-trained BERT-based model
pretrained_ner_model=TokenClassificationModel.from_pretrained("ner_en_bert")

### Make Predictions ###

In [16]:
# define the list of queries for inference
queries=[
    'we bought four shirts from the nvidia gear store in santa clara.',
    'Nvidia is a company.',
]

# make sample predictions
results=pretrained_ner_model.add_predictions(queries)

# show predictions
for query, result in zip(queries, results):
    print(f'Query : {query}')
    print(f'Result: {result.strip()}\n')
    print()

NameError: name 'pretrained_ner_model' is not defined

### Evaluate Predictions ###

To see how the model performs, we can generate predictions similar to the way we did it before and compare it with the labels. Alternatively, the `evaluate_from_file()` method enables us to evaluate the model given `text_file` and `labels_file`. Optionally, you can use the `add_confusion_matrix` to get a visual representation of the model performance. 

In [ ]:
# create a subset of our dev data
!head -n 100 $DATA_DIR/text_dev.txt > $DATA_DIR/sample_text_dev.txt
!head -n 100 $DATA_DIR/labels_dev.txt > $DATA_DIR/sample_labels_dev.txt

Now, let's generate predictions for the provided text file. If labels file is also specified, the model will evaluate the predictions and plot confusion matrix.

In [ ]:
WORK_DIR = "WORK_DIR"

# evaluate model performance on sample
pretrained_ner_model.evaluate_from_file(
    text_file=os.path.join(DATA_DIR, 'sample_text_dev.txt'),
    labels_file=os.path.join(DATA_DIR, 'sample_labels_dev.txt'),
    output_dir=WORK_DIR,
    add_confusion_matrix=True,
    normalize_confusion_matrix=True,
    batch_size=1
)

## Fine-Tune a Pre-Trained Model ##

Without specifying configuration file, NeMo will use the default configurations for the model and trainer. When fine-tuning a pre-trained NER model, we need to setup training and evaluation data before training, the dataset directory is the only required argument if the files names are `labels_dev.txt`, `labels_train.txt`, `text_dev.txt`, and `text_train.txt`. 

In [ ]:
import pytorch_lightning as pl

# setup the data dir to get class weights statistics
pretrained_ner_model.update_data_dir(DATA_DIR)

# setup train and validation Pytorch DataLoaders
pretrained_ner_model.setup_training_data()
pretrained_ner_model.setup_validation_data()

In [ ]:
# set up loss
pretrained_ner_model.setup_loss()

_Note:_ Use `class_balancing='weighted_loss'` if you want to add class weights to the `CrossEntropyLoss`. 

In [ ]:
# create a PyTorch Lightning trainer and call `fit` again
fast_dev_run=True
trainer=pl.Trainer(devices=1, accelerator='gpu', fast_dev_run=fast_dev_run)
trainer.fit(pretrained_ner_model)

_Note:_ When training a model, we can set up the model (and trainer) using a configuration file. This is not needed since the task we are performing is the same as the pre-trained model. We will train a custom token classification model in the next notebook, which will require using a configuration file. Furthermore, we are setting `fast_dev_run` to `True` for this demonstration so the trainer will run 1 training batch and 1 validation batch. For actual model training, disable the flag. 

In [ ]:
# evaluate model performance on sample
pretrained_ner_model.evaluate_from_file(
    text_file=os.path.join(DATA_DIR, 'sample_text_dev.txt'),
    labels_file=os.path.join(DATA_DIR, 'sample_labels_dev.txt'),
    output_dir=WORK_DIR,
    add_confusion_matrix=True,
    normalize_confusion_matrix=True,
    batch_size=1
)

In [ ]:
# restart the kernel
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**Well Done!** When you're ready, let's move to the [next notebook](./02_domain-specific_token_classification_model.ipynb).

![DLI Header](images/DLI_Header.png)